# Test harness for Tensorflow Serving

This notebook tests the `jedi_sith` Tensorflow Serving implementation.

In [1]:
jedi_sith_url = "http://jedi_sith:8501/v1/models/jedi_sith"

In [2]:
import math
import json
import requests
import numpy as np
import tensorflow as tf
import cv2
from io import BytesIO

In [3]:
def keras_img_to_array(img, data_format=None, dtype=None):
    # Copied from Keras 2.11.0
    if data_format is None:
        data_format = 'channels_last'
    if dtype is None:
        dtype = "float32"
    if data_format not in {"channels_first", "channels_last"}:
        raise ValueError(f"Unknown data_format: {data_format}")
    # Numpy array x has format (height, width, channel)
    # or (channel, height, width)
    # but original PIL image has format (width, height, channel)
    x = np.asarray(img, dtype=dtype)
    if len(x.shape) == 3:
        if data_format == "channels_first":
            x = x.transpose(2, 0, 1)
    elif len(x.shape) == 2:
        if data_format == "channels_first":
            x = x.reshape((1, x.shape[0], x.shape[1]))
        else:
            x = x.reshape((x.shape[0], x.shape[1], 1))
    else:
        raise ValueError(f"Unsupported image shape: {x.shape}")
    return x


In [4]:
    image_data = cv2.imread("data/debug.png", cv2.IMREAD_COLOR)
    image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)

In [5]:
img = keras_img_to_array(image_data)
img = img[:,:,:3]
img = tf.image.resize(img, [200,200])
#img = tf.image.convert_image_dtype(img, tf.float32)
img_np = img.numpy().tolist()

In [6]:
headers = {"content-type": "application/json"}
data = json.dumps({"signature_name": "serving_default", "instances": [img_np]})
json_response = requests.post(jedi_sith_url + ":predict", data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']
prediction = predictions[0]
score = prediction[0]
score

0.156882837

In [7]:
score_offset = score * 10
score_idx = round(score_offset)
score_idx

2